In [9]:
import scipy.io
import numpy as np
import pandas as pd

def load_cwru_data(mat_file_path, bearing_type, rpm, label):
    mat = scipy.io.loadmat(mat_file_path)
    
    for key in mat.keys():
        if "DE_time" in key:
            signal = mat[key].squeeze()
            break

    window_size = 1024
    step = 1024
    features = []

    for i in range(0, len(signal) - window_size, step):
        window = signal[i:i+window_size]
        feat = {
            "Max": np.max(window),
            "Min": np.min(window),
            "Mean": np.mean(window),
            "Std Dev": np.std(window),
            "RMS": np.sqrt(np.mean(window**2)),
            "Skewness": pd.Series(window).skew(),
            "Kurtosis": pd.Series(window).kurt(),
            "Crest Factor": np.max(np.abs(window)) / np.sqrt(np.mean(window**2)),
            "Form Factor": np.sqrt(np.mean(window**2)) / np.mean(np.abs(window)),
            "Label": label,
            "RPM": rpm,
            "BearingType": bearing_type
        }
        features.append(feat)

    return pd.DataFrame(features)


In [11]:
df_normal = load_cwru_data("raw/Time_Normal_1_098.mat", "Normal", rpm=1797, label=0)

df_inner_7 = load_cwru_data("raw/IR007_1_110.mat", "Inner", 1797, label=1)
df_inner_14 = load_cwru_data("raw/IR014_1_175.mat", "Inner", 1772, label=1)
df_inner_21 = load_cwru_data("raw/IR021_1_214.mat", "Inner", 1750, label=1)

df_outer_7 = load_cwru_data("raw/OR007_6_1_136.mat", "Outer", 1797, label=2)
df_outer_14 = load_cwru_data("raw/OR014_6_1_202.mat", "Outer", 1772, label=2)
df_outer_21 = load_cwru_data("raw/OR021_6_1_239.mat", "Outer", 1750, label=2)

df_ball_7 = load_cwru_data("raw/B007_1_123.mat", "Ball", 1797, label=3)
df_ball_14 = load_cwru_data("raw/B014_1_190.mat", "Ball", 1772, label=3)
df_ball_21 = load_cwru_data("raw/B021_1_227.mat", "Ball", 1750, label=3)

# Combine all data
df_all = pd.concat([
    df_normal,
    df_inner_7, df_inner_14, df_inner_21,
    df_outer_7, df_outer_14, df_outer_21,
    df_ball_7, df_ball_14, df_ball_21
], ignore_index=True)

df_all.to_csv("cwru_features.csv", index=False)
df_all.head()


,Max,Min,Mean,Std Dev,RMS,Skewness,Kurtosis,Crest Factor,Form Factor,Label,RPM,BearingType
0,0.184416,-0.198810,0.005364,0.065107,0.065328,-0.374253,-0.076941,3.043269,1.242784,0,1797,Normal
1,0.204234,-0.171065,0.010640,0.067653,0.068484,-0.202500,-0.288822,2.982204,1.233377,0,1797,Normal
2,0.204443,-0.197350,0.009911,0.062318,0.063101,-0.258258,0.161475,3.239937,1.260030,0,1797,Normal
3,0.179201,-0.182330,0.009280,0.063649,0.064322,-0.285696,-0.171557,2.834656,1.231738,0,1797,Normal
4,0.204860,-0.180244,0.010609,0.063624,0.064503,-0.115862,-0.209694,3.176002,1.232975,0,1797,Normal
